In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
from __future__ import print_function
from builtins import super
"""
Implementation of Block Neural Autoregressive Flow
http://arxiv.org/abs/1904.04676
"""
import torch.nn.functional as F
import random
import torch
torch.set_printoptions(profile="full")
import copy
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as D
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import warnings
import torchvision
print ("torch",torch.__version__)
print ("torchvision",torchvision.__version__)
warnings.filterwarnings("ignore")

import sys
import array
import math

import time
import argparse
import pprint
import scipy
from scipy import special
from functools import partial
import json
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from tqdm import tqdm
np.set_printoptions(threshold=sys.maxsize)
from sklearn.manifold import TSNE
import csv
from torch.utils import data

from tqdm import trange





torch 1.8.1
torchvision 0.2.2


In [2]:

# --------------------
# Data
# --------------------
matplotlib.use('tkagg')
class sample:
    def __init__(self, nchan ,biasmag, batch_size, topred, seed=None):
        self.biasmag=biasmag
        self.batch_size=batch_size
        self.nchan=nchan
        self.seed=seed
        self.topred=topred
    def sreturn(self):
        xvals=[]
        yvals=[]
        ychan=[]
        if (self.seed==None):
                rng = np.random
        else:
                rng = np.random.RandomState(self.seed)
        #print (-1,1,(self.batch_size,self.nchan))
        randonarr=rng.uniform(-1,1,(self.batch_size,self.nchan))*self.biasmag
        #randonarr[:,1]=1.0*randonarr[:,0]+0.0*randonarr[:,1]


        ntimep = 100+self.topred
        for bb in range(self.batch_size):


                curx=[]
                cury=[]
                prevarrand=np.array([])
                perm=np.arange(4)
                np.random.shuffle(perm)
                #print(perm)
                ychan.append(perm)
                for nn in range(self.nchan):
                        sinfac=rng.uniform(0.2,1.)*1.2
                        sinarr=np.sin(np.arange(ntimep)*sinfac)*5
                        cosfac=rng.uniform(0.2,1.)*1.2
                        cosarr=np.cos(np.arange(ntimep)*cosfac)*5
                        #print(sinfac)
                        #print(cosfac)
                        corrnoise=False
                        arrand = rng.uniform(-1,1,ntimep)
                        if corrnoise:
                                if prevarrand.size==0:
                                        arrand = rng.uniform(-1,1,ntimep)
                                else:
                                        arrand = rng.uniform(-1,1,ntimep)
                                        arrand[nn*5:]=prevarrand[0:-1*nn*5]
                                cs=np.cumsum(arrand)
                        else:
                                arrand = rng.uniform(-1,1,ntimep)
                                cs=np.cumsum(arrand+randonarr[bb][nn])+sinarr+cosarr

                        #print (sinarr) 
                        curx.append(cs[:-self.topred])
                        cury.append(cs[-self.topred:])
                        #curx.append(np.cumsum(arrand+randonarr[bb][0]))
                        prevarrand=arrand
                curx=np.array(curx)
                cury=np.array(cury)
                #print (ychan[-1])
                #print (curx[:,0])

                curx=curx[ychan[-1]]
                #curx=np.transpose(curx,ychan[-1])
                #print (curx[:,0])
                #print ()
                cury=cury[ychan[-1]]
     
                xvals.append((curx))
                yvals.append((cury))
                #print (np.array(xvals).shape)
                #print (np.array(yvals).shape)
                #plt.plot(curx)
                #plt.xlabel("steps")
                #plt.ylabel("position")
                #plt.title("randdwalk")
                #plt.show()
                #sys.exit()
                #input("Press Enter to continue...")
        return np.array(xvals, dtype="float32"),np.array(yvals, dtype="float32"),np.array(ychan, dtype="int64")







In [3]:
from os import path

class stockdata:
    def __init__(self,totsym=-1,reload=False):
        
        self.tottime=None
        self.values=None
        self.range=None
        self.corrs=None
        self.symnames=[]
        dirname="filtered/"
        alldown=os.listdir(dirname)
        exbool=path.exists("stockdata.json")
        self.corrs={}
        self.sigmadist=[]
        print (exbool)
        if (not exbool) and (not reload):
            reload=True
            print("file not found, reloading ")
        if not reload:
            self.values=pd.read_json ("stockdata.json")
            self.range = pd.read_json ("range.json")
            self.tottime=self.values.iloc[0].shape[0]
        maxsym=len(alldown)
        print("Loading...")
        for ialld,alld in enumerate (alldown):
            
            if totsym>0:
                maxsym=totsym
                if ialld>totsym:
                    break
            if ialld%500==0:
                print(ialld,"/",maxsym)
            if alld.find("corr_")!=-1:
                continue

            self.symnames.append(alld.replace(".json",""))
            if not reload:
                    continue
            curdf=pd.DataFrame(pd.read_json(dirname+alld))
            mergeframe=0.5*(curdf.loc[:,"Close"]+curdf.loc[:,"Open"])
            diffd=(curdf.loc[:,"High"]-curdf.loc[:,"Low"]) 
            
            self.sigmadist.extend(((mergeframe-curdf.loc[:,"Low"])/diffd).values.tolist())
            #print ( self.sigmadist[-1])
            #print(mergeframe)
            mergeframe.name=self.symnames[-1]
            diffd.name=self.symnames[-1]

            if (ialld == 0):
                self.values=mergeframe
                self.tottime=mergeframe.shape[0]
                self.range=diffd
            else:
                self.values=pd.concat((self.values,mergeframe),axis=1) 
                self.range=pd.concat((self.range,diffd),axis=1) 

                
            self.corrs[self.symnames[-1]]=pd.Series((pd.read_json(dirname+"corr_"+alld,typ="series")))
        if reload:

            self.values.to_json ("stockdata.json")
            self.range.to_json ("range.json")
        print("Done!")


In [4]:
#sd1=stockdata(totsym=500,reload=False)
#stda=((np.array(sd1.sigmadist)))
#stda = stda[~np.isnan(stda)]
#stdev=stda.std()

In [5]:
sd=stockdata(reload=False)
stda=((np.array(sd.sigmadist)))
stda = stda[~np.isnan(stda)]
stdev=0.18
print(stdev)

True
Loading...
0 / 5664
500 / 5664
1000 / 5664
1500 / 5664
2000 / 5664
2500 / 5664
3000 / 5664
3500 / 5664
4000 / 5664
4500 / 5664
5000 / 5664
5500 / 5664
Done!
0.18


In [6]:
class stockdatagen:
    def __init__(self, data, nchan=10, batch_size=128, lookback=30, topred=5,stdev=0.0,seed=374561):
        self.batch_size=batch_size
        self.nchan=nchan
        self.lookback=lookback
        self.topred=topred
        self.data=data
        self.seed=seed
        self.tottime=data.tottime
        self.values=data.values
        self.stdev=stdev
        self.range=data.range
        #print (self.range)
        self.corrs=data.corrs
        self.symnames=data.symnames
        print(len(self.symnames))
        if (self.seed==None):
                self.rng = np.random
        else:
                self.rng = np.random.RandomState(self.seed)

    def sreturn(self,start=None,Syms=None):
        xvalbatch=[]
        yvalbatch=[]
        ychanbatch=[]

        maxinc=self.tottime-2*(self.lookback+self.topred)
        
        if Syms==None:
            numpoints=self.batch_size
        else:
            numpoints=len(Syms)
            
        for bb in range(numpoints):
            
            if start==None:
                    starttime=int(random.randint(0, maxinc))
            else:
                    starttime=int(start)
            for icursym,cursym in enumerate(np.array(self.symnames)):

                if (icursym>(self.nchan-1)):
                        break
                xvalrange=np.abs(self.range.loc[:,cursym].iloc[starttime:starttime+self.lookback])

                xvalsigma=self.rng.normal(loc=0.0, scale=xvalrange*self.stdev)
                         

                if icursym==0:
                    xval=[(self.values.loc[:,cursym].iloc[starttime:starttime+self.lookback]).values+xvalsigma]
                    yval=[(self.values.loc[:,cursym].iloc[starttime+self.lookback:starttime+self.lookback+self.topred]).values]  
                else:

                    xval.append ((self.values.loc[:,cursym].iloc[starttime:starttime+self.lookback]).values+xvalsigma)
                    yval.append ((self.values.loc[:,cursym].iloc[starttime+self.lookback:starttime+self.lookback+self.topred]).values)
                    #ychan.append(yval.mean())
            xval=np.array(xval)
            yval=np.array(yval)


            #categ=[0.0]*xval.shape[0]

            norm=xval[:,-1]
            norm=norm.reshape(norm.shape[0],1)
            xval= ((xval)/norm)-1.0
            yval= ((yval)/norm)-1.0
            yvmeans = yval.mean(axis=1)
            perc=np.percentile(yvmeans,99)
            ychan=yvmeans>perc
            #print (ychan.sum())
            xvalbatch.append(np.array(xval))
            yvalbatch.append(np.array(yval))
            ychanbatch.append(np.array(yvmeans))

            #gain=ychanbatch.max(axis=1)-xvalbatch[:,-1]
        return(np.array (xvalbatch, dtype="float32"),np.array (yvalbatch, dtype="float32"),np.array (ychanbatch, dtype="float32"))


In [7]:
sdg=stockdatagen(sd,nchan=128, batch_size=16, lookback=30, topred=10,stdev=stdev, seed=374361)
sdg.sreturn()

2832


(array([[[ 3.88112590e-02,  6.07144609e-02,  6.40217438e-02,
           6.63063303e-02,  7.07183108e-02,  7.10200891e-02,
           8.56036171e-02,  9.07824039e-02,  8.37555900e-02,
           7.37612322e-02,  7.03669861e-02,  9.08456072e-02,
           6.45623952e-02,  8.57765302e-02,  7.85163492e-02,
           8.04234669e-02,  8.50287974e-02,  8.22574198e-02,
           7.61404261e-02,  8.17205384e-02,  7.39508867e-02,
           2.36258078e-02,  2.74819173e-02,  1.93517357e-02,
           5.11139259e-03,  8.98770522e-03,  2.02295720e-03,
          -1.95662677e-02, -1.54735669e-02,  0.00000000e+00],
         [ 4.20848846e-01,  3.87336314e-01,  4.04590368e-01,
           4.02121902e-01,  4.20118779e-01,  4.34212565e-01,
           3.64226758e-01,  3.76035184e-01,  3.56512040e-01,
           3.45345885e-01,  3.96819860e-01,  4.09527391e-01,
           3.98400396e-01,  3.31427574e-01,  2.74055660e-01,
           2.18373656e-01,  2.39003211e-01,  2.28436351e-01,
           9.77029651e-

In [8]:

class LSTM(nn.Module):
    def __init__(self,nchan=4,ntime=100,n_hidden=256,n_layers=64,npoint=5):
        super(LSTM, self).__init__()
        self.n_hidden=n_hidden
        self.n_layers=n_layers
        self.ntime=ntime+npoint
        self.nchan=nchan
        self.npoint=npoint 
        self.output_size=self.nchan*self.npoint
        self.ks=10





        self.lin = nn.Sequential(  
        nn.Linear(464, 2*400)
        )

        self.out = nn.Linear(self.n_hidden*self.npoint, self.output_size)
        
        self.out10 = nn.Linear(self.n_hidden,self.n_hidden)
        self.out11 = nn.Linear(self.n_hidden,self.nchan*self.npoint)


        self.lstm = nn.LSTM(input_size=nchan,hidden_size= self.n_hidden, num_layers=n_layers,batch_first=True)
        self.lstm1 = nn.LSTM(input_size=1,hidden_size= self.n_hidden, num_layers=n_layers,batch_first=True)
        self.lstminter = nn.LSTM(input_size=self.n_hidden,hidden_size= self.n_hidden, num_layers=n_layers,batch_first=True)

        #self.soft = nn.Sequential( nn.Softmax(dim=1))


    def forward(self, inputs):
        inputlstm=inputs[0]
        inputlstmshape=inputlstm.shape
        batchs =inputlstmshape[0]
        inputconv=inputs[1].unsqueeze(1).transpose(1, 2)


        h0c = torch.rand( self.n_layers, batchs, self.n_hidden).to('cuda')  
        c0c = torch.rand( self.n_layers, batchs, self.n_hidden).to('cuda')
        LSc,hiddenc=self.lstm1(inputconv, (h0c,c0c)) 
        print(inputconv.shape)
        print(h0c.shape)
        print(c0c.shape)

        inputlstm=inputlstm.transpose(1, 2)

        #print(LSc.shape)
        #print(inputlstm.shape)
        h0 = torch.rand ( self.n_layers, batchs, self.n_hidden).to('cuda')  
        c0 = torch.rand( self.n_layers, batchs, self.n_hidden).to('cuda')
        LS,hidden=self.lstm(inputlstm, hiddenc) 
        LS=F.relu(LS)
        LS,hidden=self.lstminter(LS, hidden) 
        #output = self.out(torch.cat((LS[:,-1,:],LSc[:,-1,:]),dim=1))
        LS=LS[:,-1,:].reshape(LS.size(0), -1)
        
        
        #output = self.out(LS)
        #return output.reshape(output.shape[0],self.nchan,self.npoint)
        LS = F.relu(self.out10(LS))
        LS = F.relu(self.out10(LS))
        output = self.out11(LS)
        return output.reshape(output.shape[0],self.nchan,self.npoint)
    
    


In [9]:

class Conv(nn.Module):
    def __init__(self,nchan=4,ntime=100,n_hidden=64,n_layers=10,npoint=5):
        super(Conv, self).__init__()
        self.n_hidden=n_hidden
        self.n_layers=n_layers
        self.ntime=ntime+npoint
        self.nchan=nchan
        self.npoint=npoint 
        self.output_size=self.nchan*self.npoint
        self.ks=10
        
        self.conv1v1_1 = nn.Sequential( 
            nn.Conv1d(self.nchan, self.nchan,kernel_size=1, padding=1),
            nn.MaxPool1d(1)
        )
        self.conv1 = nn.Sequential( 
            nn.Conv1d(int(self.nchan), int(self.nchan),kernel_size=3, padding=1),
            nn.MaxPool1d(2)
        )
        self.conv2 = nn.Sequential( 
            nn.Conv1d(int(2*self.nchan), int(self.nchan),stride=1, kernel_size=3, padding=0),
            nn.MaxPool1d(2)
        )
        self.conv3 = nn.Sequential( 
            nn.Conv1d(int(self.nchan/4), int(self.nchan),stride=1, kernel_size=3, padding=3),
            nn.MaxPool1d(1)
        )

        self.lin1 = nn.Sequential(  
        nn.Linear(16, 128)
        )
        self.lin2 = nn.Sequential(  
        nn.Linear(128, 8)
        )    
        self.lininter = nn.Sequential(  
        nn.Linear(int(self.nchan/2), self.nchan)
        )
        self.out = nn.LSTM(input_size=int(8), hidden_size=16,num_layers=8,batch_first=True)

        self.out1 = nn.Sequential( 
        nn.AdaptiveAvgPool1d(1),
        nn.Flatten()
        )



    def forward(self, inputs):
        #print(inputs.shape)

        batchs =inputs.shape[0]

        CO = F.relu(self.conv1(inputs))
        #print(CO.shape)
        #CO = F.relu(self.conv2(CO))  
        CO=F.relu(self.lin1(CO))
        CO=F.relu(self.lin2(CO))
        #print(CO.shape)
        h0 = torch.rand( 8, batchs, 16).to('cuda')  
        c0 = torch.rand( 8, batchs, 16).to('cuda')
        #CO=CO.transpose(1, 2)
        #print(CO.shape)


        #print(CO[0])
        output,hidden = self.out(CO,(h0,c0))
        #print(output.shape)
        return F.tanh(output[:,:,-1].reshape(batchs,-1))


In [10]:

trsamp={}
nchan=2832
batch_size=32
lookback=32
topred=4

plt.close('all')
n_steps=1000
nchantot=nchan
ntime=lookback+topred
nepoch=int(n_steps/20)
biasvsal=0.1
topred=topred
lateststart= sd.tottime - ntime 
regenval=False
regentrain=False
if regenval:
    tempval=(stockdatagen(sd,nchan=nchan, batch_size=1, lookback=lookback, topred=topred,stdev=stdev, seed=375363)).sreturn(start=lateststart)
    np.save("valfilex.npy",tempval[0])
    np.save("valfiley.npy",tempval[2])
if regentrain:

    temptrain=(stockdatagen(sd,nchan=nchan, batch_size=1024, lookback=lookback, topred=topred,stdev=stdev, seed=375363)).sreturn()
    np.save("trainfilex.npy",temptrain[0])
    np.save("trainfiley.npy",temptrain[2])



class datagen:
    def __init__(self,setx,sety,batch_size,seed=None):
        self.setx=setx
        self.sety=sety
        self.seed=seed
        self.batch_size=batch_size
        self.ntot=self.setx.shape[0]

        if (self.seed==None):
                self.rng = np.random
        else:
                self.rng = np.random.RandomState(self.seed)
    def sreturn(self):
        indices=self.rng.permutation(self.ntot)[:batch_size]
        return( self.setx[indices], self.sety[indices])
trainx=np.load("trainfilex.npy")
trainy=np.load("trainfiley.npy")
valx=np.load("valfilex.npy")
valy=np.load("valfiley.npy")
trsamp["train"]=datagen(trainx,trainy,batch_size)
trsamp["val"]=datagen(valx,valy,1)
print("Done",trsamp["train"].sreturn()[0].shape)

Done (32, 2832, 32)


In [11]:






symmask=None

iepoch=0



modelconv = Conv(nchan=nchantot,npoint=topred,n_hidden=256,ntime=ntime).to('cuda')
modelconv.train()

        
#criterion = nn.BCELoss()
criterion = nn.MSELoss()
optimizerconv = torch.optim.Adam(modelconv.parameters(), lr=0.0001, weight_decay=0.0)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizerconv, factor=0.5, patience=0, cooldown=40, verbose=True)

#valsamp=torch.from_numpy(fullval[0]).to('cuda')
#valsamptrue=torch.from_numpy(fullval[1]).to('cuda')
fig, axs = plt.subplots(1)
plotuple={"loss":[]}
with tqdm(total=n_steps, desc='Start step {}; Training for {} steps'.format(0, n_steps)) as pbar:
        for istep in range(n_steps):
            
                optimizerconv.zero_grad()
                curtuple = trsamp["train"].sreturn()
                curdata = torch.from_numpy(curtuple[0]).to('cuda')
                #print (curdata.shape)
                #curyvals = curtuple[1]
                outputsconv  = modelconv(curdata)
                predchantrue  = outputsconv
                #print (curchantrue.sum())
                curchantrue = torch.from_numpy(curtuple[1]).to('cuda').float()
                #pcc=predchantrue.detach().cpu().numpy()
                #perc=(np.percentile(pcc,95,axis=1)).reshape(pcc.shape[0],1)
  
                #pmask=torch.from_numpy(np.greater(pcc,perc))
                #pmask1=torch.from_numpy(np.less(pcc,perc))

                #curchpm=curchantrue[pmask].detach().cpu().numpy()
                #curchpm1=curchantrue[pmask1].detach().cpu().numpy()

                #print (curchpm.sum(),curchpm.sum()/float (curchpm.size))
                #print (curchpm1.sum(),curchpm1.sum()/float (curchpm1.size))
    
                #print(predchantrue.mean())
                #print(curchantrue.mean())
                lossconv = criterion(predchantrue,curchantrue).mean()

                lossconv.backward(retain_graph=False)
                torch.nn.utils.clip_grad_norm_(modelconv.parameters(), max_norm=0.1)
                #torch.nn.utils.clip_grad_norm_(modelconv.parameters(), 0.05)
                
                optimizerconv.step()   
                
                pbar.set_postfix(L='{:.4f}'.format(lossconv))
                pbar.update()
                #print (curyvals[predchantrue>perc])
                #axs[0].cla()
                #axs[1].cla()
                #axs[0].plot(curyvals [predchantrue>perc])
                #axs[1].plot(np.append(valsamp.cpu().numpy()[bnum,1,:],outputs.cpu().detach().numpy()[bnum,1,:]))
                scheduler.step(lossconv)
                if (istep%nepoch==0 and istep>0):


                    allsymm=np.array(sd.symnames)

                    modelconv.eval()
                    fullval=trsamp["val"].sreturn()
                    curdatav = torch.from_numpy(fullval[0]).to('cuda')


                    curchantruev = torch.from_numpy(fullval[1]).to('cuda').float()
                    predchantruev  = modelconv(curdatav)
                    print (criterion(predchantruev,curchantruev).mean())
                    #print(curdatav)
                    pccv=predchantruev.detach().cpu().numpy()
                    print(pccv.shape)

                    percv=(np.percentile(pccv,95,axis=1)).reshape(pccv.shape[0],1)

                    pmaskv=torch.from_numpy(np.greater(pccv,percv))
                    pmaskv1=torch.from_numpy(np.less(pccv,percv))
                    pmaskv=pmaskv.reshape(-1)
                    pmaskv1=pmaskv1.reshape(-1)
                    if iepoch>0:
                        print(list(set(symmask) - set(allsymm[pmaskv])))
                    symmask=allsymm[pmaskv]
                    print (allsymm[pmaskv])

                    print(curchantruev[:,pmaskv].detach().cpu().numpy().mean())
                    print(curchantruev[:,pmaskv1].detach().cpu().numpy().mean())

                    curdatatruevmeans = curchantruev
                    truepccv=curdatatruevmeans.detach().cpu().numpy()
                    print(truepccv.shape)
                    truepercv=(np.percentile(truepccv,95,axis=1)).reshape(truepccv.shape[0],1)
                    
                    truepmaskv=torch.from_numpy(np.greater(truepccv,truepercv))
                    truepmaskv1=torch.from_numpy(np.less(truepccv,truepercv))
                    
                    truepmaskv=truepmaskv.reshape(-1)
                    truepmaskv1=truepmaskv1.reshape(-1)
                    
                    print(curdatatruevmeans[:,truepmaskv].detach().cpu().numpy().mean())
                    print(curdatatruevmeans[:,truepmaskv1].detach().cpu().numpy().mean())

                    modelconv.train()
                    iepoch+=1
        
        
        plt.close()


Start step 0; Training for 1000 steps:   0%|                                | 2/1000 [00:06<52:35,  3.16s/it, L=0.0599]

Epoch     2: reducing learning rate of group 0 to 5.0000e-05.


Start step 0; Training for 1000 steps:   4%|█▎                             | 43/1000 [02:09<47:36,  2.99s/it, L=0.0499]

Epoch    43: reducing learning rate of group 0 to 2.5000e-05.


Start step 0; Training for 1000 steps:   5%|█▌                             | 51/1000 [02:33<47:42,  3.02s/it, L=0.0325]

...

Epoch   372: reducing learning rate of group 0 to 9.7656e-08.


Start step 0; Training for 1000 steps:  40%|████████████                  | 401/1000 [20:07<30:18,  3.04s/it, L=0.0270]

tensor(0.0246, device='cuda:0', grad_fn=<MeanBackward0>)
(1, 2832)
['OTEX', 'KSM', 'XPL', 'PAYS', 'ABT', 'XPO', 'FORR', 'FORD', 'DB', 'CBIO', 'XPER', 'KRNY']
['A' 'AACG' 'AAIC' 'AAL' 'AAME' 'AAON' 'AAP' 'AAPL' 'AAU' 'AAWW' 'AB'
 'ABB' 'ABC' 'ABCB' 'ABEO' 'ABEV' 'ABG' 'ABIO' 'ABM' 'ABMD' 'ABR' 'ABST'
 'AER' 'AES' 'ARE' 'ARGO' 'ARKR' 'ARL' 'AVID' 'AVK' 'BBY' 'BC' 'CBFV'
 'CZWI' 'D' 'DAC' 'DAL' 'DAN' 'DAR' 'EIM' 'EIX' 'EL' 'ELA' 'ELLO' 'ELOX'
 'ELP' 'GFED' 'GFF' 'GOL' 'GOLD' 'GOOD' 'HIG' 'HIHO' 'HIL' 'HIMX' 'HOLI'
 'HOLX' 'HURC' 'HURN' 'HUSA' 'JBL' 'JBLU' 'JBSS' 'KRO' 'KSPN' 'KSS' 'KSU'
 'KT' 'MCHP' 'MHK' 'MPV' 'MPW' 'MPWR' 'NRT' 'NS' 'NSC' 'NSEC' 'NSIT' 'NSL'
 'NTRS' 'NTUS' 'NTWK' 'NTZ' 'NUAN' 'NUE' 'NUO' 'NURO' 'NUS' 'NUV' 'NUVA'
 'NUW' 'NVAX' 'NVCN' 'NVDA' 'NVEC' 'OC' 'OCC' 'OCFC' 'OCN' 'OCSL' 'OCUP'
 'ODC' 'ODFL' 'OSIS' 'OSK' 'OSPN' 'OSTK' 'OSUR' 'PAR' 'PATK' 'PCK' 'PCM'
 'PESI' 'PETS' 'PFBC' 'PFBI' 'PFC' 'PFD' 'PFE' 'PFG' 'PML' 'PMM' 'PMX'
 'PRIM' 'SRCE' 'SRCL' 'SRDX' 'SRE' 'SRGA' 'S

Start step 0; Training for 1000 steps:  41%|████████████▎                 | 411/1000 [20:42<29:41,  3.02s/it, L=0.0280]


KeyboardInterrupt: 